importation des données brutes

In [0]:
df = spark.read.load("/databricks-datasets/airlines/part-00000", format="csv", sep=",", inferSchema=True, header=True)
display(df)

In [0]:
df.createOrReplaceTempView("airlines")
spark_df = spark.sql("SELECT * FROM airlines")
display(spark_df)

In [0]:
#enregistrement de la table bronze
spark_df.write.format("delta").mode("overwrite").saveAsTable("airlines")

In [0]:
#première modification et transformation du jeu de données
not_cancelled_flights_df = spark.sql("SELECT * FROM airlines WHERE Cancelled = 0")
display(not_cancelled_flights_df)

In [0]:
final_df = not_cancelled_flights_df.select("Dest", "Origin", "Year", "Month", "DayofMonth", "FlightNum", "Distance")
display(final_df)

In [0]:
# enregistrement de la table silver
final_df.write.format("delta").mode("overwrite").saveAsTable("airlines2")

In [0]:
#requête pour analyser les vols par destination, étape gold
from pyspark.sql.functions import count, avg

result_df = final_df.groupBy("Dest").agg(
    count("*").alias("NumberOfFlights"),
    avg("Distance").alias("AverageDistance")
).orderBy("NumberOfFlights", ascending=False)

display(result_df)

In [0]:
#enregistrement de la table gold
result_df.createOrReplaceTempView("flights_analysis")

In [0]:
#visualisation des résultats de la table gold
%sql
select * from flights_analysis